In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pandarallel import pandarallel
import regex as re
import glob
import os
pandarallel.initialize(progress_bar=True, nb_workers= 16, verbose = 0)
tqdm.pandas()

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import interpolate
import numpy as np
from scipy.ndimage import gaussian_filter1d

In [64]:
file_paths=[]
path = r'/Users/kakaoohnehose/GitHub/Bachelor-Arbeit/notebooks/classifiers/poc/lda/lda_score_csv' # path
print(path)
#file_paths = [f for f in os.listdir(path) if f.endswith('.csv')]
for file in  glob.glob(os.path.join(path , "*.csv")):
    file_paths.append(file)
print("loading from :",len(file_paths),"files")
while '.csv' in file_paths:
    file_paths.remove('.csv')
#safe all file names as cleaned keys
regexp_1 = re.compile(r"(?P<size>\d{3,}(?=_topics.*))")
regexp_2 = re.compile(r"(?P<topics>\d{1,2}(?=\.csv))")
subst = ""
file_k_size = []
file_k_topic = []
i = 1
for str in file_paths:
    m = regexp_1.search(str)
    m2 = regexp_2.search(str)
    file_k_size.append(m.group("size"))
    file_k_topic.append(m2.group("topics"))




/Users/kakaoohnehose/GitHub/Bachelor-Arbeit/notebooks/classifiers/poc/lda/lda_score_csv
loading from : 38 files


In [66]:

file_k_topic = map(int, file_k_topic)
file_k_size = map(int, file_k_size)

In [68]:
df = pd.concat(map(pd.read_csv,file_paths),keys=[int(file_k_topic) ,file_k_size])

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'map'

In [101]:
df["topics"] = list(file_keys)[1]

In [62]:
df

Unnamed: 0                 class  topic  count
<map object at 0x142cd9bd0> 0            0            ErnÃ¤hrung   16.0     18
                            1            1            ErnÃ¤hrung   17.0   3717
                            2            2            ErnÃ¤hrung    NaN   1643
                            3            3        Landwirtschaft   16.0     64
                            4            4        Landwirtschaft   17.0   4007
                            5            5        Landwirtschaft    NaN   1466
                            6            6               Medizin   16.0    119
                            7            7               Medizin   17.0   3996
                            8            8               Medizin    NaN   1358
                            9            9  Umweltwissenschaften   16.0     41
                            10          10  Umweltwissenschaften   17.0   3745
                            11          11  Umweltwissenschaften    NaN   1536
<map object at 0x142cda3e0> 0            0            ErnÃ¤hrung    0.0   1138
                            1            1            ErnÃ¤hrung    3.0      1
                            2            2            ErnÃ¤hrung    NaN    202
                            3            3        Landwirtschaft    0.0   1172
                            4            4        Landwirtschaft    NaN    149
                            5            5               Medizin    0.0   1255
                            6            6               Medizin    NaN    152
                            7            7  Umweltwissenschaften    0.0   1129
                            8            8  Umweltwissenschaften    NaN    235